In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from functools import partial

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

import torchvision
from torchvision import datasets
from torchvision import transforms

import numpy as np

In [2]:
DEVICE = torch.device("cpu" if torch.cuda.device_count() < 1 else "cuda:0")
batch_size = 4
CLASSES = 2
DIR = os.getcwd()
EPOCHS = 5
N_TRAIN_EXAMPLES = batch_size * 30
N_VALID_EXAMPLES = batch_size * 10

print(f"Device: {DEVICE}\nBatch size: {batch_size}\nClasses: {CLASSES}\n\
Dir: {DIR}\nEpochs: {EPOCHS}\n\
Number of training examples: {N_TRAIN_EXAMPLES}\n\
Number of validation examples: {N_VALID_EXAMPLES}")

Device: cuda:0
Batch size: 4
Classes: 2
Dir: /home/bernardo/github/sex-age-estimation/backup-bia/patch-1
Epochs: 5
Number of training examples: 120
Number of validation examples: 40


In [3]:
model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

In [4]:
from torchvision import transforms as T
img_size = 224

transform = T.Compose([
                T.Resize((img_size,img_size)),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

In [5]:
from torch.utils.data import DataLoader
from configs import Inputs
from utils.augmentations import get_transforms
from utils.data import FullRadiographSexDataset

val_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                       fold_nums=Inputs().val_folds,
                                       transforms=get_transforms(Inputs(), subset=["train"]))

val_dataloader = DataLoader(val_dataset,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=0)

train_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                         fold_nums=Inputs().train_folds,
                                         transforms=get_transforms(Inputs(), subset=["train"]))

train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=0)

Using only horizontal flip augmentation.
Using only horizontal flip augmentation.


In [6]:
config = {
    "lr": tune.loguniform(1e-5, 1e-1),
    "optimizer_name": tune.choice(["Adam", "RMSprop", "SGD"])
}

In [9]:
def objective(config):

    # Gerar o modelo
    #model = define_model(trial).to(DEVICE)
    model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model.to(DEVICE)

    # Gerar optimizer
    optimizer_name = config['optimizer_name']
    lr = config["lr"]
    
    print("opt_name:", optimizer_name, "\nlr:", lr)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, valid_loader = train_dataloader, val_dataloader

    criterion = F.nll_loss

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            # print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
            #                                     running_loss / epoch_steps))
            running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(val_dataloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1
        
        accuracy= correct / total

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=loss.cpu(), accuracy=accuracy)

    return accuracy

In [ ]:
scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=EPOCHS,
        grace_period=1,
        reduction_factor=2)

reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])

result = tune.run(partial(objective),
                  resources_per_trial={"cpu": 0, "gpu": 1},
                  config=config,
                  num_samples=N_TRAIN_EXAMPLES,
                  scheduler=scheduler,
                  progress_reporter=reporter,
)

(raylet) [2022-12-27 03:48:54,051 E 21876 21899] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_03-43-11_833949_21808 is over 95% full, available space: 15308992512; capacity: 455568355328. Object creation will fail if spilling is required.


== Status ==
Current time: 2022-12-27 03:48:52 (running for 00:00:00.27)
Memory usage on this node: 4.3/15.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_03-48-51
Number of trials: 16/120 (15 PENDING, 1 RUNNING)
+-----------------------+----------+------------------+-------------+------------------+
| Trial name            | status   | loc              |          lr | optimizer_name   |
|-----------------------+----------+------------------+-------------+------------------|
| objective_60d4d_00000 | RUNNING  | 172.17.0.2:22322 | 0.000366285 | RMSprop          |
| objective_60d4d_00001 | PENDING  |                  | 4.64575e-05 | RMSprop          |
| objective_60d4d_00002 | PENDING  |                  | 8.81425e-05 | RMSprop          |
| objective_60d4d_00003 | PEND

(raylet) [2022-12-27 03:49:04,057 E 21876 21899] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_03-43-11_833949_21808 is over 95% full, available space: 15308980224; capacity: 455568355328. Object creation will fail if spilling is required.


== Status ==
Current time: 2022-12-27 03:49:04 (running for 00:00:12.75)
Memory usage on this node: 5.8/15.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_03-48-51
Number of trials: 17/120 (16 PENDING, 1 RUNNING)
+-----------------------+----------+------------------+-------------+------------------+
| Trial name            | status   | loc              |          lr | optimizer_name   |
|-----------------------+----------+------------------+-------------+------------------|
| objective_60d4d_00000 | RUNNING  | 172.17.0.2:22322 | 0.000366285 | RMSprop          |
| objective_60d4d_00001 | PENDING  |                  | 4.64575e-05 | RMSprop          |
| objective_60d4d_00002 | PENDING  |                  | 8.81425e-05 | RMSprop          |
| objective_60d4d_00003 | PEND

(raylet) [2022-12-27 03:49:14,063 E 21876 21899] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_03-43-11_833949_21808 is over 95% full, available space: 15308951552; capacity: 455568355328. Object creation will fail if spilling is required.


...

== Status ==
Current time: 2022-12-27 03:56:29 (running for 00:07:37.23)
Memory usage on this node: 5.8/15.6 GiB
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 4.000: 162.83470153808594 | Iter 2.000: 85.3678970336914 | Iter 1.000: 23.790725708007812
Resources requested: 0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_03-48-51
Number of trials: 20/120 (16 PENDING, 1 RUNNING, 3 TERMINATED)
+-----------------------+------------+------------------+-------------+------------------+-------------------+------------+----------------------+
| Trial name            | status     | loc              |          lr | optimizer_name   | loss              |   accuracy |   training_iteration |
|-----------------------+------------+------------------+-------------+------------------+-------------------+------------+----------------------|
| objective_60d4d_00003 | RUNNING    | 172.17.0.2:22582 | 0.00437527 

(raylet) [2022-12-27 03:56:34,318 E 21876 21899] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_03-43-11_833949_21808 is over 95% full, available space: 15064576000; capacity: 455568355328. Object creation will fail if spilling is required.


== Status ==
Current time: 2022-12-27 03:56:39 (running for 00:07:47.24)
Memory usage on this node: 5.8/15.6 GiB
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 4.000: 162.83470153808594 | Iter 2.000: 85.3678970336914 | Iter 1.000: 23.790725708007812
Resources requested: 0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_03-48-51
Number of trials: 20/120 (16 PENDING, 1 RUNNING, 3 TERMINATED)
+-----------------------+------------+------------------+-------------+------------------+-------------------+------------+----------------------+
| Trial name            | status     | loc              |          lr | optimizer_name   | loss              |   accuracy |   training_iteration |
|-----------------------+------------+------------------+-------------+------------------+-------------------+------------+----------------------|
| objective_60d4d_00003 | RUNNING    | 172.17.0.2:22582 | 0.00437527 

(raylet) [2022-12-27 03:56:44,324 E 21876 21899] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-27_03-43-11_833949_21808 is over 95% full, available space: 15064576000; capacity: 455568355328. Object creation will fail if spilling is required.


== Status ==
Current time: 2022-12-27 03:56:49 (running for 00:07:57.26)
Memory usage on this node: 5.8/15.6 GiB
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 4.000: 162.83470153808594 | Iter 2.000: 85.3678970336914 | Iter 1.000: 23.790725708007812
Resources requested: 0/8 CPUs, 1.0/1 GPUs, 0.0/8.3 GiB heap, 0.0/4.15 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-27_03-48-51
Number of trials: 20/120 (16 PENDING, 1 RUNNING, 3 TERMINATED)
+-----------------------+------------+------------------+-------------+------------------+-------------------+------------+----------------------+
| Trial name            | status     | loc              |          lr | optimizer_name   | loss              |   accuracy |   training_iteration |
|-----------------------+------------+------------------+-------------+------------------+-------------------+------------+----------------------|
| objective_60d4d_00003 | RUNNING    | 172.17.0.2:22582 | 0.00437527 